In [1]:
import pandas as pd
import numpy as np
import datetime

import psycopg2
import pymysql

from datetime import timedelta

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
#current_refresh = pd.read_excel("/home/jupyter-ngenia-jackline/Limits_refresh_summary_20220901_multiple_products_rec.xlsx")
current_refresh = pd.read_excel("/home/jupyter-ngenia-jackline/Limits_refresh_summary_20220922_multiple_products_rec_1_new.xlsx")
#current_refresh = current_refresh[['store_number', 'new_final_21_limit', 'updated_7_limit', 'updated_1_limit']]
#current_refresh = current_refresh.rename(columns={'updated_7_limit':'new_final_7_limit','updated_1_limit':'new_final_1_limit'})

#current_refresh = current_refresh.fillna(0)

current_refresh.head()

,Unnamed: 0.1,Unnamed: 0,mobile_number,store_number,sum_3_months_trx_val,avg_3_months_trx_val,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,IDM_factor_21,IDM_factor_7,IDM_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,old_final_21_limit,old_final_7_limit,old_final_1_limit,blacklist_flag,total_final_21_limit,previous_21_limit,previous_7_limit,previous_1_limit,model_version,created_at,record_added_to_warehouse_on_timestamp,new_final_21_limit_1,new_final_7_limit_1,new_final_1_limit_1,old_src_crdt_score,zeroized_7,zeroized_1,Average Repayments by Due Date as a %age of Principal amount,Average Repayments by Rollover End Date as a %age of Principal amount,updated_7_limit,updated_1_limit
0,0,0,254723173634,786152,5430.000,1810.000,3700.000,2022-08-25,2022-09-20,27,4,0.150,No_rules_relaxed,2.000,Yes,1.000,494024.000,Approve,0.000,True,NaN,1,601.000,30.000,12698.000,9206.000,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.000,1.000,written-off_default,NaN,12698.000,2017-12-18,0.000,0.000,1738.000,0,0,0.500,0.170,0.170,1.000,1.000,1.000,Band 1,Band 2,0.000,0.000,0.000,0,0,2,0.500,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,1,0,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,NaN,0,0,0.000,0.000,0,0
1,1,1,254723916436,165978,2454577.510,818192.503,1005490.230,2022-08-23,2022-09-21,30,30,1.000,No_rules_relaxed,1.000,Yes,1.000,20417564.000,Approve,0.000,True,NaN,1,601.000,30.000,20000.000,12800.000,2017-12-18,2018-01-17,2019-12-31,2018-01-17,713.000,1.000,written-off_default,NaN,20000.000,2017-12-18,0.000,0.000,1738.000,0,0,0.500,0.170,0.170,1.000,1.000,1.000,Band 7,Band 2,0.250,0.150,0.150,0,1,1,0.500,0.125,0.075,0.075,102274.063,61364.438,61364.438,0.000,61364.438,61364.438,0,0,0,1,304500,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,NaN,0,0,0.000,0.000,0,0
2,2,2,254720475133,745142,463108.240,154369.413,194218.800,2022-08-22,2022-09-21,31,30,0.970,No_rules_relaxed,1.000,Yes,1.000,22893884.000,Reject,0.000,True,NaN,1,601.000,30.000,28684.000,0.000,2017-12-21,2018-01-20,2019-12-31,2018-01-20,710.000,1.000,written-off_default,NaN,28684.000,2017-12-21,0.000,0.000,1735.000,0,0,0.350,0.120,0.120,0.700,0.706,0.706,Band 7,Band 2,0.250,0.150,0.150,0,1,1,0.500,0.087,0.053,0.053,13507.324,8172.498,8172.498,0.000,8172.498,8172.498,0,0,0,1,27200,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,NaN,0,0,0.000,0.000,0,0
3,3,3,254724756423,978161,1100.000,366.667,0.000,NaT,NaT,0,0,0.000,No_rules_relaxed,NaN,0,0.000,13261352.000,Approve,0.000,True,NaN,2,601.000,30.000,28600.000,0.000,2018-01-07,2018-02-06,2018-12-31,2018-02-06,328.000,1.000,written-off_default,NaN,28680.000,2017-12-19,1.000,0.500,1718.000,0,0,0.500,0.170,0.170,1.000,1.000,1.000,Band 1,Band 2,0.000,0.000,0.000,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,1,0,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,NaN,0,0,0.000,0.000,0,0
4,4,4,25472299

In [4]:
current_refresh[(current_refresh['new_final_21_limit_1'] > 0) & (current_refresh['updated_7_limit'] == 0)].shape

(443, 86)

In [5]:
def updated_new_7_limit(df):
    
    updated_7_limit = df['updated_7_limit']
    new_final_21_limit_1 = df['new_final_21_limit_1']
    old_final_7_limit = df['old_final_7_limit']
    
    if new_final_21_limit_1 > 0 and updated_7_limit == 0:
        return old_final_7_limit
    else:
        return updated_7_limit
    
    
def updated_new_1_limit(df):
    
    updated_1_limit = df['updated_1_limit']
    new_final_21_limit_1 = df['new_final_21_limit_1']
    old_final_1_limit = df['old_final_1_limit']
    
    if new_final_21_limit_1 > 0 and updated_1_limit == 0:
        return old_final_1_limit
    else:
        return updated_1_limit

In [6]:
current_refresh['updated_new_21_limit'] = current_refresh['new_final_21_limit_1']
current_refresh['updated_new_7_limit'] = current_refresh.apply(lambda x: updated_new_7_limit(x), axis = 1)
current_refresh['updated_new_1_limit'] = current_refresh.apply(lambda x: updated_new_1_limit(x), axis = 1)

print(current_refresh[['updated_new_21_limit', 'updated_new_7_limit', 'updated_new_1_limit']].sum())

updated_new_21_limit    204942500
updated_new_7_limit     442234200
updated_new_1_limit     444016400
dtype: int64


In [7]:
current_refresh.to_excel("/home/jupyter-ngenia-jackline/Limits_refresh_summary_20220922_multiple_products_rec_2_new.xlsx")